In [12]:
import os

import pandas as pd
import numpy as np


from imblearn.under_sampling import TomekLinks as TomekLinks
from sklearn.model_selection import RandomizedSearchCV as RSCV
from sklearn.model_selection import GridSearchCV as GSCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.utils.class_weight import compute_sample_weight
from xgboost import XGBRFClassifier as xgbrfc
from xgboost import XGBClassifier as xgbc

import joblib

In [13]:
trainset = pd.read_csv('ori_feature_set.csv', index_col = 0)
X = trainset.drop('Y', axis = 1)
y = trainset.Y

tl = TomekLinks()
X, y = tl.fit_resample(X, y)

# smt_set = pd.read_csv('smt_feature_set.csv', index_col = 0)
# X_smt = smt_set.drop('Y', axis = 1)
# y_smt = smt_set.Y

In [14]:
def rf_model_selection(X, y):
    weights = compute_sample_weight("balanced", y)
    params = {
        'n_estimators': [i for i in range(100, 1100, 100)],
        'criterion': ['gini', 'entropy'],
    }
    rf = RandomForestClassifier(class_weight='balanced')
    gs = GSCV(rf,
              param_grid = params,
              n_jobs = -1)
    gs_results = gs.fit(X, y, sample_weight = weights)
    return gs_results

In [15]:
# random search model parameters combination
# rf_gs_results = rf_model_selection(X, y)

In [16]:
# print('best rf score: ', rf_gs_results.best_score_)

In [77]:
weights = compute_sample_weight("balanced", y)
params_base = {
    'objective': 'multi:softmax',
    'eval_metric': 'logloss',
    'n_estimators': 540,
#     'max_depth': 1,
#     'min_child_weight': 1,
#     'gamma': 0.001,
#     'subsample': 0.2,
#     'reg_alpha': 60,
#     'eta': 1e-4,
}
params = {
#     'n_estimators': [i for i in range(530, 580, 10)],
#     'max_depth': [i for i in range(1, 4)],
#     'min_child_weight':[1,2,3,4,5,6],
#     'gamma':[0.0001, 0, 0.01,0.001],
#     'subsample':[0.1,0.2,0.3,0.4,0.5],
#     'reg_alpha':[i for i in range(10, 110, 10)],
#     'eta':[1e-4, 1e-5, 1e-6, 1e-3],
}

# params_another = {
#     'objective': 'multi:softprob',
#     'eval_metric': 'mlogloss',
#     'n_estimators': 700,
#     'max_depth': 4,
#     'min_child_weight': 5,
#     'gamma': 5,
#     'reg_alpha': 0.1,
#     'eta': 0.871
# }

In [78]:
xgb = xgbc(**params_base)
gs = GSCV(xgb,
          param_grid = params,
          n_jobs = -1)
xgb_gs_results = gs.fit(X, y, sample_weight = weights)

/Users/shyanechang/opt/anaconda3/envs/py39/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/shyanechang/opt/anaconda3/envs/py39/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/shyanechang/opt/anaconda3/envs/py39/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To

In [79]:
print('xgb gs best score', xgb_gs_results.best_score_)
print(xgb_gs_results.best_params_)

xgb gs best score 0.5345585748792271
{}


In [61]:
# model save
joblib.dump(xgb_gs_results.best_estimator_, 'xgb_gscv_best_model.pkl')

['xgb_gscv_best_model.pkl']

In [64]:
testset = pd.read_csv('test.csv', index_col = 0)
X_test = testset[X.columns]
X_test

,R5,T8,G2,G1,T7,G3,T11,O3,R6,O2,R7,T1,G4,O4,T30
ID,,,,,,,,,,,,,,,
1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0
2,0,0,1,1,0,0,0,0,0,0,1,0,0,1,1
3,0,0,1,1,0,0,0,1,1,0,0,0,1,0,0
4,0,0,1,1,0,0,0,0,0,1,1,0,1,0,0
5,0,0,0,1,0,1,0,0,0,0,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,0,0,1,1,0,0,1,0,1,0,0,0,0,1,0
297,0,0,1,1,0,0,0,0,0,0,0,0,1,1,1
298,0,0,1,0,0,0,0,0,0,0,1,1,1,1,0


In [65]:
prediction = xgb_gs_results.best_estimator_.predict_proba(X_test)
prediction

/Users/shyanechang/opt/anaconda3/envs/py39/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


array([[0.33560184, 0.32888365, 0.3355145 ],
       [0.32989234, 0.34004298, 0.33006468],
       [0.33560184, 0.32888365, 0.3355145 ],
       [0.33560184, 0.32888365, 0.3355145 ],
       [0.32989234, 0.34004298, 0.33006468],
       [0.33560184, 0.32888365, 0.3355145 ],
       [0.32989234, 0.34004298, 0.33006468],
       [0.32989234, 0.34004298, 0.33006468],
       [0.3356742 , 0.32895458, 0.3353713 ],
       [0.3356742 , 0.32895458, 0.3353713 ],
       [0.33560184, 0.32888365, 0.3355145 ],
       [0.33560184, 0.32888365, 0.3355145 ],
       [0.32989234, 0.34004298, 0.33006468],
       [0.33560184, 0.32888365, 0.3355145 ],
       [0.32989234, 0.34004298, 0.33006468],
       [0.33560184, 0.32888365, 0.3355145 ],
       [0.3356742 , 0.32895458, 0.3353713 ],
       [0.32989234, 0.34004298, 0.33006468],
       [0.32989234, 0.34004298, 0.33006468],
       [0.33560184, 0.32888365, 0.3355145 ],
       [0.32989234, 0.34004298, 0.33006468],
       [0.33560184, 0.32888365, 0.3355145 ],
       [0.

In [66]:
xgb_submission = pd.DataFrame(prediction, columns = ['C1','C2','C3'], index = testset.index)
xgb_submission

,C1,C2,C3
ID,,,
1,0.335602,0.328884,0.335514
2,0.329892,0.340043,0.330065
3,0.335602,0.328884,0.335514
4,0.335602,0.328884,0.335514
5,0.329892,0.340043,0.330065
...,...,...,...
296,0.335674,0.328955,0.335371
297,0.329892,0.340043,0.330065
298,0.335674,0.328955,0.335371


In [67]:
xgb_submission.to_csv('xgb_submission.csv')